In [ ]:
from typing import Tuple
import cv2
import numpy
import numpy.typing
import matplotlib.pyplot as plt
import os
from dataclasses import dataclass

In [ ]:
def read_images(source_dir: str) -> Tuple[list[cv2.Mat], list[str]]:
    available_image_files = os.listdir(source_dir)
    return [cv2.imread(source_dir + "\\" + img) for img in available_image_files], available_image_files

In [ ]:
#fazer exibição ao invés de armazenamento
def save_filtered_images(dest_dir:str, imgs: list[cv2.Mat], names: list[str]):
    for img, name in zip(imgs, names):
        plt.imsave(dest_dir+"\\"+name, img, cmap='gray')

In [ ]:
def display_filtered_images(imgs: list[cv2.Mat], names: list[str]):
    for img, name in zip(imgs, names):
        plt.imshow(img, cmap='gray')
        plt.show()

In [ ]:
# Função auxiliar p vídeo
def test_with_video(processing_func, params):
    import time
    video = cv2.VideoCapture(0)
    win = cv2.namedWindow('video')
    #deixar mais seguro
    while True:
        ret, frame = video.read()
        result_image = processing_func(frame, params)
        cv2.imshow('video', result_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(0.01)

    video.release()
    cv2.destroyAllWindows()

## Filtra parte amarela

In [ ]:
@dataclass
class YellowFilteringParams:
    kernel_size: int
    std_dev: int
    yellow_min: numpy.ndarray
    yellow_max: numpy.ndarray
    def set_yellow_min_h(self, val):
        self.yellow_min[0] = val
    def set_yellow_min_s(self, val):
        self.yellow_min[1] = val
    def set_yellow_min_v(self, val):
        self.yellow_min[2] = val
    def set_yellow_max_h(self, val):
        self.yellow_min[0] = val
    def set_yellow_max_s(self, val):
        self.yellow_min[1] = val
    def set_yellow_max_v(self, val):
        self.yellow_min[2] = val
    def set_kernel_size(self, val):
        if val % 2 == 0:
            return
        self.kernel_size = val
    def set_std_dev(self, val):
        self.std_dev = val        

In [ ]:
working_params = YellowFilteringParams(3, 0, numpy.array([29, 93, 0]), numpy.array([52, 256,256]))

In [ ]:
def filter_yellow(img: cv2.Mat, params: YellowFilteringParams) -> cv2.Mat:
    img = cv2.GaussianBlur(img, (params.kernel_size, params.kernel_size), params.std_dev)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV_FULL)
    yellow_min = params.yellow_min  # valores HSV mínimos
    yellow_max = params.yellow_max  # valroes HSV máximos
    yellow_region = numpy.where(numpy.all(numpy.logical_and(
                numpy.less_equal(yellow_min, img),
                numpy.less_equal(img, yellow_max)), axis=2),
            255*numpy.ones(img.shape[:2]), 0)
    return yellow_region

In [ ]:
test_with_video(filter_yellow, working_params)

In [ ]:
p = working_params
#yellow_min=array([29, 93,  0]), yellow_max=array([ 52, 277, 277])
# p.yellow_min = numpy.array([29, 93,  0])
# p.yellow_max = numpy.array([ 52, 256, 256])
img_list, img_names = read_images(".\\Marco de pouso")
yellow_regions = [filter_yellow(img, p) for img in img_list]

# save_filtered_images(".\\Filtered landing pads", yellow_regions, img_names)
# display_filtered_images(yellow_regions, img_names)

## Extrai contornos da região amarela

In [ ]:
@dataclass
class ContourExtractParams:
    size_threshold: int
    
def extract_contours(yellow_region : cv2.Mat, params : ContourExtractParams) -> tuple:
    #fazer conversão certa
    yellow_region_u8 = yellow_region.astype(numpy.uint8)
    #retornar informação de hierarquia p identificação do + tbm?
    yellow_contours, hierarchy = cv2.findContours(yellow_region_u8, cv2.RETR_TREE,
        cv2.CHAIN_APPROX_NONE)
    return tuple(filter(lambda cont: len(cont) > params.size_threshold, yellow_contours))
    # return yellow_contours
    
def draw_contours(yellow_region, size_threshold):
    yellow_contours = extract_contours(yellow_region, size_threshold)
    yellow_contour_image = numpy.zeros_like(yellow_region)
    cv2.drawContours(yellow_contour_image, yellow_contours, -1, 255, 2)
    return yellow_contour_image

In [ ]:
yellow_contour_images = [draw_contours(yr, 100) for yr in yellow_regions]
yellow_contours = [extract_contours(yr, 100) for yr in yellow_regions]
# save_filtered_images("Contour images", yellow_contour_images, img_names)

In [ ]:
test_with_video(lambda x, params: draw_contours(filter_yellow(x, params[0]), params[1]), [working_params, 100])

In [ ]:
cv2.destroyAllWindows()

## Fit de elipse a um blob - transformada de Hough

In [ ]:
def fit_ellipse(yellow_contours, residue_thresh):
    is_ellipse = []
    coef_list = []
    centroid = []
    for cont in yellow_contours:
        positions = numpy.float64(cont.copy())
        avg_pos = numpy.mean(positions, axis=0)
        positions -= avg_pos
        x = positions[:, 0, 0]
        y = positions[:, 0, 1]

        if not max(abs(x)) < 1e-5:
            x /= max(abs(x))
        if not max(abs(y)) < 1e-5:
            y /= max(abs(y))

        try:
            coefs, resid, rank, sing = numpy.linalg.lstsq(
                numpy.column_stack((x**2, x*y, y**2)), 
                numpy.ones_like(x), rcond=None)
        except numpy.linalg.LinAlgError:
            return False, [], numpy.array([-1,-1])

        is_ellipse.append(resid <= residue_thresh)
        coef_list.append(coefs)
        centroid.append(avg_pos)

    return is_ellipse, coef_list, centroid

def identify_ellipse(yellow_contours, residue_thresh, size):
    is_ellipse, coef_list, centroid = fit_ellipse(yellow_contours, residue_thresh)
    display_image = numpy.zeros(size)
    for is_el, cent in zip(is_ellipse, centroid):
        cv2.drawContours(display_image, yellow_contours, -1, 255, 2)
        cv2.circle(display_image, [int(cent[0][0]), int(cent[0][1])], 5, 255, -1)
    return display_image

In [ ]:
test_with_video(lambda x, params: identify_ellipse(
    extract_contours(
        filter_yellow(x, params[0]), params[1]
    ), params[2], params[3]), [working_params, 200, 100, [480, 640]])

In [ ]:
# ellipses = [fit_ellipse(yc, 100) for yc in yellow_contours]
ellipses = [identify_ellipse(yc, 100, yellow_contour_images[0].shape) for yc in yellow_contours]
save_filtered_images("Ellipses", ellipses, img_names)

In [ ]:
@dataclass
class EllipseTrackerParams:
    yellow_filtering: YellowFilteringParams
    contour_size_threshold: int
    residue_thresh: float
    dist_threshold: float
    alpha: float

class EllipseTracker:
    def __init__(self, params : EllipseTrackerParams):
        self.last_center_candidates = None
        self.params = params
        self.best_center = None
    def __call__(self, new_image):
        is_ellipse, coef_list, centroid = \
            fit_ellipse(extract_contours(
                filter_yellow(new_image,
                    self.params.yellow_filtering),
                self.params.contour_size_threshold),
                self.params.residue_thresh
        )

        candidate_ellipse_centroids = [c for i, c in enumerate(centroid) if is_ellipse[i]]

        if self.last_center_candidates is not None and len(self.last_center_candidates) > 0:
            candidate_distances = []
            for c in candidate_ellipse_centroids:
                distances_to_last = [numpy.linalg.norm(c - last) for last in self.last_center_candidates]
                candidate_distances.append(min(distances_to_last))
            if len(candidate_distances) > 0:
                best_index = numpy.argmin(candidate_distances)
                self.last_center_candidates = candidate_ellipse_centroids
                if self.best_center is None:
                    self.best_center = candidate_ellipse_centroids[best_index]
                else:
                    self.best_center = self.params.alpha * candidate_ellipse_centroids[best_index] \
                                    + (1 - self.params.alpha) * self.best_center
                
                return self.best_center
        
        self.last_center_candidates = candidate_ellipse_centroids
        return None
    
    def draw_ellipse(self, new_image, radius):
        display_image = new_image.copy()
        center = self.__call__(new_image)
        if center is not None:
            cv2.circle(display_image, numpy.int32(center[0]), radius, (255, 255, 255), -1)
        return display_image
        

In [ ]:
ellipse_params = EllipseTrackerParams(working_params, 100, 100, 50, 0.5)

In [ ]:
#mudar formato do display
cv2.namedWindow("Trackbars")
cv2.createTrackbar("L-H", "Trackbars", 0, 180, ellipse_params.yellow_filtering.set_yellow_min_h)
cv2.createTrackbar("L-S", "Trackbars", 0, 255, ellipse_params.yellow_filtering.set_yellow_min_s)
cv2.createTrackbar("L-V", "Trackbars", 0, 255, ellipse_params.yellow_filtering.set_yellow_min_v)
cv2.createTrackbar("U-H", "Trackbars", 0, 180,  ellipse_params.yellow_filtering.set_yellow_max_h)
cv2.createTrackbar("U-S", "Trackbars", 0, 255, ellipse_params.yellow_filtering.set_yellow_max_s)
cv2.createTrackbar("U-V", "Trackbars", 0, 255, ellipse_params.yellow_filtering.set_yellow_max_v)
cv2.createTrackbar("Kernel", "Trackbars", 0, 50, ellipse_params.yellow_filtering.set_kernel_size)
cv2.createTrackbar("std dev", "Trackbars", 0, 20, ellipse_params.yellow_filtering.set_std_dev)

tracker = EllipseTracker(ellipse_params)

function_to_test = lambda x, radius: tracker.draw_ellipse(x, radius)
test_with_video(function_to_test, 5)

$$
ax^2 + bxy + cy^2 -1 = 0
$$
$$
\therefore y = \frac{-bx \pm \sqrt{(bx)^2 -4c(ax^2-1)}}{2c}
$$

In [ ]:
cv2.destroyAllWindows()

## Reconhecimento do mais

In [ ]:
yellow_image = yellow_regions[0]
plt.imshow(yellow_image, cmap='gray')
type(yellow_image)

In [ ]:
contours = extract_contours(yellow_image, ContourExtractParams(20))

In [ ]:
cont = contours[0]
approx = cv2.approxPolyDP(cont, 0.01*cv2.arcLength(cont, True), True)
approx

In [ ]:
approx.ravel()

In [ ]:
@dataclass
class PlusIdParams:
    contour_epsilon:float
    minimal_area:float
    vertex_count_tolerance:int

def identify_plus(contour, params):
    approx_polygonal_vertices = cv2.approxPolyDP(contour, params.contour_epsilon * cv2.arcLength(contour, True), True)
    M = cv2.moments(contour)
    area = M['m00']
    if area > params.minimal_area:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        if abs(len(approx_polygonal_vertices) - 12) < params.vertex_count_tolerance and not cv2.isContourConvex(contour):
            return cX, cY
    return None

def identify_pluses(contours, params):
    return [identify_plus(cont, params) for cont in contours]

def draw_plus(rgb_img, plusidparams):
    yellow_region = filter_yellow(rgb_img, working_params)
    contours = extract_contours(yellow_region, ContourExtractParams(20))
    pluses = identify_pluses(contours, plusidparams)
    display_image = rgb_img.copy()
    for plus, cont in zip(pluses, contours):
        if plus is not None:
            cv2.drawContours(display_image, [cont], -1, (0,0,0), 5)
            cv2.circle(display_image, plus, 5, (0, 0, 0), -1)
    return display_image

In [ ]:
test_with_video(draw_plus, PlusIdParams(0.01, 400, 2))

In [ ]:
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
#import imutils

cap = cv2.VideoCapture(0)
def nothing(x):
    pass

cv2.namedWindow("Trackbars")
cv2.createTrackbar("L-H", "Trackbars", 25, 180, nothing)
cv2.createTrackbar("L-S", "Trackbars", 60, 255, nothing)
cv2.createTrackbar("L-V", "Trackbars", 140, 255, nothing)
cv2.createTrackbar("U-H", "Trackbars", 65, 180, nothing)
cv2.createTrackbar("U-S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U-V", "Trackbars", 255, 255, nothing)
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    lh = cv2.getTrackbarPos("L-H", "Trackbars")
    ls = cv2.getTrackbarPos("L-S", "Trackbars")
    lv = cv2.getTrackbarPos("L-V", "Trackbars")
    uh = cv2.getTrackbarPos("U-H", "Trackbars")
    us = cv2.getTrackbarPos("U-S", "Trackbars")
    uv = cv2.getTrackbarPos("U-V", "Trackbars")


    low_red = np.array([lh, ls, lv])
    up_red = np.array([uh, us, uv])
    mask = cv2.inRange(hsv, low_red, up_red)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #cnts = imutils.grab_contours(contours)
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        # compute the center of the contour
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cX = int((M["m10"] / M["m00"]))
            cY = int((M["m01"] / M["m00"]))
        area = cv2.contourArea(cnt)
        if area > 400:
            if len(approx) > 10 and len(approx) < 13:
                cv2.putText(frame, "Cruz", (x,y), font, 2, (0,0,0))
            cv2.drawContours(frame, [approx], -1, (0,0,0), 5)


    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

## Landing Pad Detector teste

In [1]:
from landing_pad_detector import *
import numpy
import cv2
from typing import List

In [2]:
yellow_filter_params = YellowFilterParams(
            numpy.array([20, 93, 0]),
            numpy.array([52, 256,256])
)
yellow_filter_trackbar_params = [
    TrackbarParams("L-H", 20 , 180, yellow_filter_params.set_yellow_min_h),
    TrackbarParams("L-S", 93 , 255, yellow_filter_params.set_yellow_min_s),
    TrackbarParams("L-V", 0  , 255, yellow_filter_params.set_yellow_min_v),
    TrackbarParams("U-H", 52 , 180, yellow_filter_params.set_yellow_max_h),
    TrackbarParams("U-S", 256, 255, yellow_filter_params.set_yellow_max_s),
    TrackbarParams("U-V", 256, 255, yellow_filter_params.set_yellow_max_v)
]

contour_extract_params = ContourExtractParams(50)
contour_extract_trackbar_params = [
    TrackbarParams("SizeThr", 50, 500, contour_extract_params.set_size_threshold)
]

ellipse_fit_params = EllipseFitParams(10)
ellipse_fit_trackbar_params = [
    TrackbarParams("ResiThr", 10, 100, ellipse_fit_params.set_residue_threshold)
]

plus_identify_params = PlusIdParams(0.01, 400, 2)
plus_identify_trackbar_params = [
    TrackbarParams("eps", 10, 200, plus_identify_params.set_contour_epsilon),
    TrackbarParams("minArea", 400, 1000, plus_identify_params.set_minimal_area),
    TrackbarParams("VerCTol", 2, 3, plus_identify_params.set_vertex_count_tolerance)
]

center_estimate_params = CenterEstimateParams(20)
center_estimate_trackbar_params = [
    TrackbarParams("Tol", 20, 50, center_estimate_params.set_tolerance)
]

tracker = Tracker(TrackerParams(0.8, 5, 50))
tracker_trackbar_params = [
    TrackbarParams("Alpha", 80, 100, tracker.params.set_alpha),
    TrackbarParams("MaxFwoD", 5, 10, tracker.params.set_max_frames_wo_detection),
    TrackbarParams("MaxDeltaPos", 50, 100, tracker.params.set_max_delta_position)
]

In [11]:
test_with_video(display_yellow, yellow_filter_params, trackbar_params=yellow_filter_trackbar_params)

In [12]:
test_with_video(display_contours, [yellow_filter_params,
                                        contour_extract_params], trackbar_params=contour_extract_trackbar_params)

In [13]:
test_with_video(display_ellipse, [yellow_filter_params,
                                        contour_extract_params,
                                        ellipse_fit_params], trackbar_params=ellipse_fit_trackbar_params)

In [14]:
test_with_video(display_plus, [yellow_filter_params,
                                        contour_extract_params,
                                        plus_identify_params], trackbar_params=plus_identify_trackbar_params)

In [16]:
test_with_video(display_center, [yellow_filter_params,
                                        contour_extract_params,
                                        plus_identify_params,
                                        ellipse_fit_params,
                                        center_estimate_params], trackbar_params=center_estimate_trackbar_params)

In [4]:
test_with_video(display_tracker, [yellow_filter_params,
                                        contour_extract_params,
                                        plus_identify_params,
                                        ellipse_fit_params,
                                        center_estimate_params,
                                        tracker], trackbar_params=tracker_trackbar_params)

In [47]:
cv2.destroyAllWindows()